In [69]:
import pandas as pd
import numpy as np

import re

In [70]:
# allows for viewing of all columns
pd.set_option("display.max.columns", None)

In [71]:
data = '/Users/memanuel0109/documents/Github/Capstone_Project_Two/data/train.csv'

In [72]:
df = pd.read_csv(data)

In [73]:
df

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_Caucasian,race_AfricanAmerican,gender_Female,age_[70-80),age_[60-70),age_[50-60),age_[80-90),age_[40-50),payer_code_?,payer_code_MC,payer_code_HM,payer_code_SP,payer_code_BC,medical_specialty_?,medical_specialty_InternalMedicine,medical_specialty_Emergency/Trauma,medical_specialty_Family/GeneralPractice,medical_specialty_Cardiology,diag_1_428,diag_1_414,diag_1_786,diag_2_276,diag_2_428,diag_2_250,diag_2_427,diag_3_250,diag_3_401,diag_3_276,diag_3_428,max_glu_serum_None,A1Cresult_None,metformin_No,repaglinide_No,nateglinide_No,chlorpropamide_No,glimepiride_No,acetohexamide_No,glipizide_No,glyburide_No,tolbutamide_No,pioglitazone_No,rosiglitazone_No,acarbose_No,miglitol_No,troglitazone_No,tolazamide_No,examide_No,citoglipton_No,insulin_No,glyburide-metformin_No,glipizide-metformin_No,glimepiride-pioglitazone_No,metformin-rosiglitazone_No,metformin-pioglitazone_No,change_No,diabetesMed_Yes,readmitted
0,14,41,0,11,0,0,0,6,True,False,False,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,0
1,2,30,0,12,0,0,1,9,True,False,True,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,False,True,True,True,True,True,False,True,1
2,5,66,0,22,1,0,2,9,True,False,True,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,1
3,3,63,0,8,0,0,0,8,True,False,True,False,False,True,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,1
4,5,40,0,6,0,0,1,9,True,False,True,False,False,False,True,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,2,85,0,12,0,0,0,9,True,False,False,False,True,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,False,True,0
24996,5,35,2,15,0,0,0,9,False,True,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,1
24997,3,10,1,23,1,0,0,4,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,True,True,True,False,True,1
24998,8,69,3,41,1,0,0,9,True,False,False,False,True,F

# Cleaning Data

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 65 columns):
 #   Column                                    Non-Null Count  Dtype
---  ------                                    --------------  -----
 0   time_in_hospital                          25000 non-null  int64
 1   num_lab_procedures                        25000 non-null  int64
 2   num_procedures                            25000 non-null  int64
 3   num_medications                           25000 non-null  int64
 4   number_outpatient                         25000 non-null  int64
 5   number_emergency                          25000 non-null  int64
 6   number_inpatient                          25000 non-null  int64
 7   number_diagnoses                          25000 non-null  int64
 8   race_Caucasian                            25000 non-null  bool 
 9   race_AfricanAmerican                      25000 non-null  bool 
 10  gender_Female                             25000 non-null  

### Create days in hopital column

In [75]:
# renamed the column to make it more clear that these values represent days spent in the hospital
df.rename({'time_in_hospital':'days_in_hospital'}, axis=1, inplace=True)

In [76]:
df.days_in_hospital.describe()

count    25000.000000
mean         4.395640
std          2.991165
min          1.000000
25%          2.000000
50%          4.000000
75%          6.000000
max         14.000000
Name: days_in_hospital, dtype: float64

### Create race column

In [77]:
# converted boolean column into categorical column for descriptive statistics purposes

In [78]:
df['race_Caucasian'].replace(True, 'Caucasian', inplace=True)

In [79]:
df['race_AfricanAmerican'].replace(True, 'African_American', inplace=True)

In [80]:
race_columns = ['race_Caucasian', 'race_AfricanAmerican']

In [81]:
df[race_columns] = df[race_columns].replace(False, '')

In [82]:
df['race'] = df['race_Caucasian'] + df['race_AfricanAmerican']

In [83]:
df['race'] = df['race'].replace('', 'Other')

In [84]:
df.drop(race_columns, axis = 1, inplace = True)

### Create gender column

In [85]:
# converted boolean column into categorical column for descriptive statistics purposes

In [86]:
def recode_gender(gender):
    """boolean vales into gender values"""
    
    if gender == True:
        return 'Female'
        
    elif gender == False:
        return 'Male'
        
    else:
        return 'Unknown'

In [87]:
df['gender'] = df.gender_Female.apply(recode_gender)

In [88]:
df.drop('gender_Female', axis=1, inplace=True)

### Create age column

In [89]:
# converted boolean column into categorical column for descriptive statistics purposes

In [90]:
age_columns = ['age_[70-80)', 'age_[60-70)', 'age_[40-50)', 'age_[80-90)', 'age_[50-60)']

In [91]:
[df[col].replace({True:str(col), False:''}, inplace=True) for col in age_columns]

[None, None, None, None, None]

In [92]:
df['age'] = df[age_columns].apply(lambda row: ''.join(row.values.astype(str)), axis=1)

In [93]:
df.drop(age_columns, axis = 1, inplace = True)

In [94]:
# renamed items in columns for cleaner look

In [95]:
age_renames = ['age70-80', 'age60-70', 'age40-50', 'age80-90', 'age50-60']

In [96]:
df['age'] = df['age'].replace(age_columns, age_renames)

### Create payer code column

In [97]:
# converted boolean column into categorical column for descriptive statistics purposes

In [98]:
paycode_columns = ['payer_code_?', 'payer_code_MC', 'payer_code_HM', 'payer_code_SP', 'payer_code_BC']

In [99]:
[df[col].replace({True:str(col), False:''}, inplace=True) for col in paycode_columns]

[None, None, None, None, None]

In [100]:
df['paycode'] = df[paycode_columns].apply(lambda row: ''.join(row.values.astype(str)), axis=1)

In [101]:
df.drop(paycode_columns, axis = 1, inplace = True)

### Create medical specialty column

In [102]:
# converted boolean column into categorical column for descriptive statistics purposes

In [103]:
specialty_columns = ['medical_specialty_?', 'medical_specialty_InternalMedicine',
                     'medical_specialty_Emergency/Trauma', 'medical_specialty_Family/GeneralPractice',
                     'medical_specialty_Cardiology']

In [104]:
[df[col].replace({True:str(col), False:''}, inplace=True) for col in specialty_columns]

[None, None, None, None, None]

In [105]:
df['specialty'] = df[specialty_columns].apply(lambda row: ''.join(row.values.astype(str)), axis=1)

In [106]:
df.drop(specialty_columns, axis = 1, inplace = True)

In [107]:
specialty_renames = ['Unknown_specialty', 'Internal_Medicine', 'Emergency/Trauma_Medicine', 'Family/General_Medicine',
                  'Cardiology']

In [108]:
df['specialty'].replace(specialty_columns, specialty_renames, inplace = True)

### Create diagnoses columns

In [109]:
# converted boolean column into categorical column for descriptive statistics purposes

In [110]:
diagnoses_columns = ['diag_1_428', 'diag_1_414', 'diag_1_786', 'diag_2_276', 'diag_2_428', 'diag_2_250', 'diag_2_427',
                     'diag_3_250', 'diag_3_401', 'diag_3_276', 'diag_3_428']

In [111]:
# looked up the ICD-9 coding for these diagnoses

In [112]:
diagnoses_renames = ['Dx1_Heart_Failure', 'Dx1_Other_Heart_Disease', 'Dx1_Respiratory_sxs', 'Dx2_pH/Fluid_Imbalance',
                     'Dx2_Heart_Failure', 'Dx2_Diabetes_Mellitus', 'Dx2_Cardiac Dysrhythmias', 'Dx3_Diabetes_Mellitus', 
                     'Dx3_Essential_Hypertension', 'Dx3_pH/Fluid_Imbalance', 'Dx3_Heart_Failure']

In [113]:
df.rename(columns = {'diag_1_428':'Dx1_Heart_Failure', 'diag_1_414':'Dx1_Other_Heart_Disease', 'diag_1_786':'Dx1_Respiratory_sxs', 
                     'diag_2_276':'Dx2_pH/Fluid_Imbalance', 'diag_2_428':'Dx2_Heart_Failure', 'diag_2_250':'Dx2_Diabetes_Mellitus',
                     'diag_2_427':'Dx2_Cardiac Dysrhythmias','diag_3_250':'Dx3_Diabetes_Mellitus', 
                     'diag_3_401':'Dx3_Essential_Hypertension', 'diag_3_276':'Dx3_pH/Fluid_Imbalance', 'diag_3_428':'Dx3_Heart_Failure'}, inplace=True)

### Labs and Meds

In [114]:
df.columns

Index(['days_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'Dx1_Heart_Failure',
       'Dx1_Other_Heart_Disease', 'Dx1_Respiratory_sxs',
       'Dx2_pH/Fluid_Imbalance', 'Dx2_Heart_Failure', 'Dx2_Diabetes_Mellitus',
       'Dx2_Cardiac Dysrhythmias', 'Dx3_Diabetes_Mellitus',
       'Dx3_Essential_Hypertension', 'Dx3_pH/Fluid_Imbalance',
       'Dx3_Heart_Failure', 'max_glu_serum_None', 'A1Cresult_None',
       'metformin_No', 'repaglinide_No', 'nateglinide_No', 'chlorpropamide_No',
       'glimepiride_No', 'acetohexamide_No', 'glipizide_No', 'glyburide_No',
       'tolbutamide_No', 'pioglitazone_No', 'rosiglitazone_No', 'acarbose_No',
       'miglitol_No', 'troglitazone_No', 'tolazamide_No', 'examide_No',
       'citoglipton_No', 'insulin_No', 'glyburide-metformin_No',
       'glipizide-metformin_No', 'glimepiride-pioglitazone_No',
       'metformin-rosiglitazone_No

In [115]:
# inverted "no" columns to better represent whether the lab test was done

In [116]:
lab_columns = ['max_glu_serum_None', 'A1Cresult_None',
       'metformin_No', 'repaglinide_No', 'nateglinide_No', 'chlorpropamide_No',
       'glimepiride_No', 'acetohexamide_No', 'glipizide_No', 'glyburide_No',
       'tolbutamide_No', 'pioglitazone_No', 'rosiglitazone_No', 'acarbose_No',
       'miglitol_No', 'troglitazone_No', 'tolazamide_No', 'examide_No',
       'citoglipton_No', 'insulin_No', 'glyburide-metformin_No',
       'glipizide-metformin_No', 'glimepiride-pioglitazone_No',
       'metformin-rosiglitazone_No', 'metformin-pioglitazone_No', 'change_No']

In [117]:
# labeled mapped out columns as "complete"

In [118]:
for column in lab_columns:
    df[column + ' complete'] = ~df[column]

In [119]:
df.drop(lab_columns, axis=1, inplace=True)

In [120]:
df.columns

Index(['days_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'Dx1_Heart_Failure',
       'Dx1_Other_Heart_Disease', 'Dx1_Respiratory_sxs',
       'Dx2_pH/Fluid_Imbalance', 'Dx2_Heart_Failure', 'Dx2_Diabetes_Mellitus',
       'Dx2_Cardiac Dysrhythmias', 'Dx3_Diabetes_Mellitus',
       'Dx3_Essential_Hypertension', 'Dx3_pH/Fluid_Imbalance',
       'Dx3_Heart_Failure', 'diabetesMed_Yes', 'readmitted', 'race', 'gender',
       'age', 'paycode', 'specialty', 'max_glu_serum_None complete',
       'A1Cresult_None complete', 'metformin_No complete',
       'repaglinide_No complete', 'nateglinide_No complete',
       'chlorpropamide_No complete', 'glimepiride_No complete',
       'acetohexamide_No complete', 'glipizide_No complete',
       'glyburide_No complete', 'tolbutamide_No complete',
       'pioglitazone_No complete', 'rosiglitazone_No complete',
       'acarbose_No com

In [121]:
# replaced string format for cleaner look

In [122]:
updated_columns = df.columns.str.replace('_No complete','')

In [123]:
final_columns = updated_columns.str.replace('_None complete','')

In [124]:
df.columns = final_columns

In [125]:
# converted all boolean columns into integers

In [126]:
df = df.applymap(lambda x: 1 if x == True else x)

In [127]:
df = df.applymap(lambda x: 0 if x == False else x)

In [128]:
df

,days_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,Dx1_Heart_Failure,Dx1_Other_Heart_Disease,Dx1_Respiratory_sxs,Dx2_pH/Fluid_Imbalance,Dx2_Heart_Failure,Dx2_Diabetes_Mellitus,Dx2_Cardiac Dysrhythmias,Dx3_Diabetes_Mellitus,Dx3_Essential_Hypertension,Dx3_pH/Fluid_Imbalance,Dx3_Heart_Failure,diabetesMed_Yes,readmitted,race,gender,age,paycode,specialty,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change
0,14,41,0,11,0,0,0,6,0,0,0,0,0,0,0,0,1,0,0,1,0,Caucasian,Male,age50-60,payer_code_SP,Unknown_specialty,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,30,0,12,0,0,1,9,0,0,0,0,0,0,0,0,0,0,0,1,1,Caucasian,Female,age50-60,payer_code_SP,Unknown_specialty,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1
2,5,66,0,22,1,0,2,9,0,0,0,0,0,0,0,0,0,0,0,1,1,Caucasian,Female,age80-90,payer_code_MC,Unknown_specialty,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,63,0,8,0,0,0,8,0,1,0,0,0,0,0,0,0,0,0,1,1,Caucasian,Female,age50-60,payer_code_?,Unknown_specialty,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5,40,0,6,0,0,1,9,0,0,0,0,0,0,0,0,1,0,0,0,0,Caucasian,Female,age80-90,payer_code_?,Family/General_Medicine,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,2,85,0,12,0,0,0,9,0,0,0,0,0,0,0,0,0,1,0,1,0,Caucasian,Male,age60-70,payer_code_SP,Unknown_specialty,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
24996,5,35,2,15,0,0,0,9,0,0,0,0,0,0,0,0,0,0,1,1,1,African_American,Female,age60-70,payer_code_?,Cardiology,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24997,3,10,1,23,1,0,0,4,0,0,0,0,0,1,0,0,0,0,0,1,1,Caucasian,Male,age70-80,payer_code_MC,Unknown_specialty,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
24998,8,69,3,41,1,0,0,9,0,0,0,0,0,0,0,0,0,0,0,1,1,Caucasian,Male,age60-70,payer_code_HM,Unknown_specialty,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1


### Medications

In [129]:
df.columns

Index(['days_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'Dx1_Heart_Failure',
       'Dx1_Other_Heart_Disease', 'Dx1_Respiratory_sxs',
       'Dx2_pH/Fluid_Imbalance', 'Dx2_Heart_Failure', 'Dx2_Diabetes_Mellitus',
       'Dx2_Cardiac Dysrhythmias', 'Dx3_Diabetes_Mellitus',
       'Dx3_Essential_Hypertension', 'Dx3_pH/Fluid_Imbalance',
       'Dx3_Heart_Failure', 'diabetesMed_Yes', 'readmitted', 'race', 'gender',
       'age', 'paycode', 'specialty', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosigli

In [130]:
medications = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone']

In [131]:
for item in medications:
     df['med_' + item] = df[item] 

In [132]:
df.drop(medications, axis=1, inplace=True)

In [133]:
[df[col].value_counts() for col in df]

[3     4379
 2     4290
 4     3506
 1     3412
 5     2436
 6     1862
 7     1394
 8     1033
 9      717
 10     580
 11     435
 12     379
 13     300
 14     277
 Name: days_in_hospital, dtype: int64,
 1      817
 43     719
 44     614
 45     581
 38     541
       ... 
 99       2
 103      2
 113      1
 126      1
 102      1
 Name: num_lab_procedures, Length: 108, dtype: int64,
 0    11445
 1     5083
 2     3160
 3     2310
 6     1225
 4     1041
 5      736
 Name: num_procedures, dtype: int64,
 12    1467
 15    1424
 11    1420
 13    1413
 14    1411
       ... 
 81       1
 66       1
 61       1
 68       1
 72       1
 Name: num_medications, Length: 71, dtype: int64,
 0     20831
 1      2172
 2       899
 3       479
 4       258
 5       143
 6        78
 7        34
 8        22
 9        19
 11       15
 10       13
 13        7
 14        7
 16        3
 17        3
 21        3
 12        3
 18        2
 24        2
 20        1
 19        1
 36        1
 34  

In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 52 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   days_in_hospital              25000 non-null  int64 
 1   num_lab_procedures            25000 non-null  int64 
 2   num_procedures                25000 non-null  int64 
 3   num_medications               25000 non-null  int64 
 4   number_outpatient             25000 non-null  int64 
 5   number_emergency              25000 non-null  int64 
 6   number_inpatient              25000 non-null  int64 
 7   number_diagnoses              25000 non-null  int64 
 8   Dx1_Heart_Failure             25000 non-null  int64 
 9   Dx1_Other_Heart_Disease       25000 non-null  int64 
 10  Dx1_Respiratory_sxs           25000 non-null  int64 
 11  Dx2_pH/Fluid_Imbalance        25000 non-null  int64 
 12  Dx2_Heart_Failure             25000 non-null  int64 
 13  Dx2_Diabetes_Mel

In [135]:
# identified any duplicate rows

In [136]:
df.duplicated().value_counts()

False    25000
dtype: int64